In [2]:
import os
import cv2
import keras
import numpy as np
import tensorflow as tf
import PIL.Image as Image
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image
from keras.layers import Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
train_path = 'Images/Training' 
test_path = 'Images/Testing'
valid_path = 'Images/Validation'

In [4]:
train_batches = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    train_path, target_size = (224,224), batch_size = 5 )
validation_batches = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    valid_path, target_size = (224,224), batch_size = 5 , shuffle = False)
test_batches = ImageDataGenerator(preprocessing_function = keras.applications.mobilenet.preprocess_input).flow_from_directory(
    test_path, target_size = (224,224), batch_size = 5, shuffle = False)

Found 444 images belonging to 2 classes.
Found 122 images belonging to 2 classes.
Found 128 images belonging to 2 classes.


In [5]:
mobile = tf.keras.applications.mobilenet.MobileNet()

In [6]:
x = mobile.layers[-6].output
x = tf.keras.layers.Flatten()(x)
output = Dense(units = 2 , activation = 'softmax')(x)

In [7]:
model = keras.Model(inputs = mobile.input , outputs = output)

In [8]:
for layer in model.layers[:-23]:
    layer.trainable = False

In [9]:
model.compile(optimizer= Adam (learning_rate = 0.001) , loss = 'binary_crossentropy' , metrics = ['accuracy'])

In [10]:
validation_batches.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [11]:
model.fit(x = train_batches , validation_data = validation_batches , epochs = 3 , verbose = 2 )

Epoch 1/3
89/89 - 22s - loss: 1.7785 - accuracy: 0.8626 - val_loss: 5.0834 - val_accuracy: 0.6475 - 22s/epoch - 247ms/step
Epoch 2/3
89/89 - 19s - loss: 0.3684 - accuracy: 0.9730 - val_loss: 0.1239 - val_accuracy: 0.9836 - 19s/epoch - 219ms/step
Epoch 3/3
89/89 - 20s - loss: 0.5449 - accuracy: 0.9685 - val_loss: 0.1896 - val_accuracy: 0.9836 - 20s/epoch - 227ms/step


In [12]:
test_labels = test_batches.classes
test_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
predictions = model.predict( x = test_batches , verbose = 0).argmax(axis=-1)

In [14]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [17]:
cm = confusion_matrix(y_true = test_labels , y_pred = predictions)

In [18]:
cm

array([[63,  0],
       [ 0, 65]], dtype=int64)

In [123]:
# test images = 128 , correctly predicted = 127  , wrongly predicted = 1

In [19]:
model.save('stamp_recog.h5')